<a href="https://colab.research.google.com/github/jaafar19/NLP/blob/main/TextSummarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import textwrap
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

In [13]:
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [1]:
!wget -nc https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv


--2025-04-02 14:04:33--  https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv
Resolving lazyprogrammer.me (lazyprogrammer.me)... 172.67.213.166, 104.21.23.210, 2606:4700:3031::6815:17d2, ...
Connecting to lazyprogrammer.me (lazyprogrammer.me)|172.67.213.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5085081 (4.8M) [text/csv]
Saving to: ‘bbc_text_cls.csv’

bbc_text_cls.csv    100%[===================>]   4.85M  --.-KB/s    in 0.1s    

2025-04-02 14:04:34 (50.9 MB/s) - ‘bbc_text_cls.csv’ saved [5085081/5085081]



In [4]:
df = pd.read_csv('bbc_text_cls.csv')

In [5]:
df.head()

,text,labels
0,Ad sales boost Time Warner profit\n\nQuarterly...,business
1,Dollar gains on Greenspan speech\n\nThe dollar...,business
2,Yukos unit buyer faces loan claim\n\nThe owner...,business
3,High fuel prices hit BA's profits\n\nBritish A...,business
4,Pernod takeover talk lifts Domecq\n\nShares in...,business


In [7]:
doc = df[df.labels == 'business']['text'].sample(random_state=42)
doc

,text
480,Christmas sales worst since 1981\n\nUK retail ...


In [10]:
def wrap(x):
  return textwrap.fill(x, replace_whitespace=False, fix_sentence_endings=True)

In [11]:
print (wrap(doc.iloc[0]))

Christmas sales worst since 1981

UK retail sales fell in December,
failing to meet expectations and making it by some counts the worst
Christmas since 1981.

Retail sales dropped by 1% on the month in
December, after a 0.6% rise in November, the Office for National
Statistics (ONS) said.  The ONS revised the annual 2004 rate of growth
down from the 5.9% estimated in November to 3.2%. A number of
retailers have already reported poor figures for December.  Clothing
retailers and non-specialist stores were the worst hit with only
internet retailers showing any significant growth, according to the
ONS.

The last time retailers endured a tougher Christmas was 23 years
previously, when sales plunged 1.7%.

The ONS echoed an earlier
caution from Bank of England governor Mervyn King not to read too much
into the poor December figures.  Some analysts put a positive gloss on
the figures, pointing out that the non-seasonally-adjusted figures
showed a performance comparable with 2003. The Novembe

In [14]:
sents = nltk.sent_tokenize(doc.iloc[0].split("\n", 1)[1])

In [15]:
featurizer = TfidfVectorizer(stop_words=stopwords.words('english'), norm='l1')

In [20]:
x = featurizer.fit_transform(sents)

In [21]:
def get_sentence_score(tfidf_row):
  x = tfidf_row[tfidf_row != 0]
  return x.mean()

In [23]:
scores = np.zeros(len(sents))
for i in range(len(sents)):
  score = get_sentence_score(x[i,:])
  scores[i]= score

In [24]:
sort_idx = np.argsort(-scores)

In [25]:
print ("Generated Summary:")
for i in sort_idx[:5]:
  print (wrap("%.2f: %s" % (scores[i], sents[i])))

Generated Summary:
0.14: A number of retailers have already reported poor figures for
December.
0.13: However, reports from some High Street retailers highlight the
weakness of the sector.
0.12: The ONS revised the annual 2004 rate of growth down from the
5.9% estimated in November to 3.2%.
0.10: "Our view is the Bank of England will keep its powder dry and
wait to see the big picture."
0.10: And a British Retail Consortium survey found that Christmas 2004
was the worst for 10 years.


In [26]:
doc.iloc[0].split('\n',1)[0]

'Christmas sales worst since 1981'

In [28]:
def summarize(text):
  sents = nltk.sent_tokenize(text)
  x = featurizer.fit_transform(sents)
  scores = np.zeros(len(sents))
  for i in range(len(sents)):
    score = get_sentence_score(x[i,:])
    scores[i]= score
  sort_idx = np.argsort(-scores)

  print ("Generated Summary:")
  for i in sort_idx[:5]:
    print (wrap("%.2f: %s" % (scores[i], sents[i])))

In [30]:
doc =df[df.labels == 'entertainment']['text'].sample(random_state=1234)
summarize(doc.iloc[0].split("\n", 1)[1])

Generated Summary:
0.33: It will be held at the Staples Center.
0.10: This year the Grammys have been dominated by rap star Kanye
West, who is in contention for 10 awards.
0.10: US comedian Ellen Degeneres and singer Christine Milian will
present awards at the event.
0.09: U2 are nominated twice for their recent single Vertigo,
including a nomination for best rock song.
0.08: Last week Grammy producers announced the show will be hosted by
rap star and Chicago actress Queen Latifah.


In [31]:
doc.iloc[0].split("\n", 1)[1]

"\nIrish rock band U2 are to play live at the Grammy Awards presentation in the US next month, organisers have said.\n\nOther acts to play include soul singer Alicia Keys, country singer Tim McGraw and punk band Green Day at the event on 13 February in Los Angeles. U2 are nominated twice for their recent single Vertigo, including a nomination for best rock song. This year the Grammys have been dominated by rap star Kanye West, who is in contention for 10 awards. US comedian Ellen Degeneres and singer Christine Milian will present awards at the event. Last week Grammy producers announced the show will be hosted by rap star and Chicago actress Queen Latifah. It will be held at the Staples Center. U2 had number one success in the album charts on both sides of the Atlantic in November when their latest studio album, How to Dismantle an Atomic Bomb, topped the US and UK charts. The band, who are also dominated for best international album at this year's Brit Awards, are to undertake a major

In [32]:
a="The change in the commodity price is the key basis of market transactions and economic management, so it is necessary to make an accurate prediction of the dynamic commodity price. \n However, due to the influences of various hidden factors, the commodity price usually varies frequently and fluctuates violently with obvious nonlinear characteristics. \nTherefore, it is difficult to obtain accurate and robust prediction results only using the historical data of the price itself. \nTo this end, this paper proposes a hybrid multi-factor price forecasting method based on factor selection and fusion with the CNN-LSTM and XGBoost network. Firstly, after we collect the data we calculated using some analytical indicator (RSA, CMF, EMA, ROC, DPO, …), secondly we apply DC framework and calculate DC parameter like aTmv, OSVOS and BM, and after extracting the features we preprocessing the tasks. \nFinally, we start training by trying the algorithm and put hyper¬-parameter and data by testing and evaluation.\nNow, after finishing evaluation we can conclude the advantage of using XGBoost is faster than CNN-LSTM in training with big dataset especially when we take a look at accuracy for all datasets, we have a higher and better accuracy with large number of transactions for XGBoost.\nIn our future work, we will work the model with another algorithm with more analytical indicator to get perfect accuracy and more precisely.\nSo, it is a promising multi-factor forecasting method and can be widely used in other fields. \nConsidering that different exogenous variables have different influences on prices, how to effectively use this characteristic to obtain more accurate prediction results is a direction that will be worthy of more research in the future."

In [33]:
summarize(a)

Generated Summary:
0.12: So, it is a promising multi-factor forecasting method and can be
widely used in other fields.
0.09: Therefore, it is difficult to obtain accurate and robust
prediction results only using the historical data of the price itself.
0.09: In our future work, we will work the model with another
algorithm with more analytical indicator to get perfect accuracy and
more precisely.
0.09: Finally, we start training by trying the algorithm and put
hyper¬-parameter and data by testing and evaluation.
0.07: The change in the commodity price is the key basis of market
transactions and economic management, so it is necessary to make an
accurate prediction of the dynamic commodity price.


In [34]:
print(wrap(a))

The change in the commodity price is the key basis of market
transactions and economic management, so it is necessary to make an
accurate prediction of the dynamic commodity price. 
 However, due to
the influences of various hidden factors, the commodity price usually
varies frequently and fluctuates violently with obvious nonlinear
characteristics. 
Therefore, it is difficult to obtain accurate and
robust prediction results only using the historical data of the price
itself. 
To this end, this paper proposes a hybrid multi-factor price
forecasting method based on factor selection and fusion with the CNN-
LSTM and XGBoost network.  Firstly, after we collect the data we
calculated using some analytical indicator (RSA, CMF, EMA, ROC, DPO,
…), secondly we apply DC framework and calculate DC parameter like
aTmv, OSVOS and BM, and after extracting the features we preprocessing
the tasks. 
Finally, we start training by trying the algorithm and put
hyper¬-parameter and data by testing and eva